In [ ]:
"""
Exemplo do algoritmo pelo ChatGPT!
"""

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def gaussian_kernel(x, y, bandwidth):
    return np.exp(-np.linalg.norm(x - y) ** 2 / (2 * (bandwidth ** 2)))

def compute_bandwidths(data, centroids, m):
    N, p = data.shape
    K = centroids.shape[0]
    bandwidths = np.zeros((K, p))
    for k in range(K):
        for j in range(p):
            numerator = 0
            denominator = 0
            for i in range(N):
                u_ik = (1 / np.linalg.norm(data[i] - centroids[k])) ** (2 / (m - 1))
                numerator += u_ik * (data[i, j] - centroids[k, j]) ** 2
                denominator += u_ik
            bandwidths[k, j] = np.sqrt(numerator / denominator)
    return bandwidths

def gkfcm(data, K, m=2, max_iter=100, tol=1e-5):
    N, p = data.shape
    # Initialize membership matrix randomly
    U = np.random.dirichlet(np.ones(K), size=N)
    centroids = np.dot(U.T, data) / U.sum(axis=0)[:, None]

    for iteration in range(max_iter):
        bandwidths = compute_bandwidths(data, centroids, m)
        prev_U = np.copy(U)
        
        for i in range(N):
            for k in range(K):
                numerator = np.sum([(U[i, c] ** m) * gaussian_kernel(data[i], centroids[c], bandwidths[c].mean()) for c in range(K)])
                U[i, k] = (1 / gaussian_kernel(data[i], centroids[k], bandwidths[k].mean())) ** (1 / (m - 1)) / numerator

        centroids = np.dot(U.T, data) / U.sum(axis=0)[:, None]
        
        if np.linalg.norm(U - prev_U) < tol:
            break

    return U, centroids, bandwidths

# Test the implementation
if __name__ == "__main__":
    # Generate some test data
    from sklearn.datasets import make_blobs

    data, _ = make_blobs(n_samples=300, centers=3, n_features=2, random_state=42)

    # Apply the GKFCM algorithm
    K = 3
    U, centroids, bandwidths = gkfcm(data, K)

    print("Membership matrix:\n", U)
    print("Centroids:\n", centroids)
    print("Bandwidths:\n", bandwidths)